In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import quantumsolver as qs

# help(qs.fewnucleonEmulator)

# Vectorized version of the fewnucleonEmulator
vfewnucleonEmulator = np.vectorize(qs.fewnucleonEmulator)


In [95]:
# LaTeX font
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'serif'
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
font_size = 12

cD_grid = np.linspace(-2.5, 2.5, 100)
cE_grid = np.linspace(-1., 1., 100)

# Create a 2-dimensional grid
cD_eval, cE_eval = np.meshgrid(cD_grid,cE_grid)


In [97]:

def constraint_plotter(cD_eval, cE_eval, efterror=False):

    E4He_eval, Rp4He_eval, _, E3He_eval, E3H_eval, _, fT3H_eval = vfewnucleonEmulator(cD_eval, cE_eval)
            
    fig, ax = plt.subplots()

    x = 1

    mean_E4He = np.mean(E4He_eval) # -28.296
    mean_Rp4He = np.mean(Rp4He_eval) # 1.4552
    mean_E3H = np.mean(E3H_eval) # -8.482
    mean_fT3H = np.mean(fT3H_eval) # 1129.6

    sigma_E3H = 0.015
    sigma_E4He = 0.005
    sigma_Rp4He = 0.0062
    sigma_fT3H = 3.

    if efterror:
        sigma_E3H = np.sqrt(0.015 + (1 * 0.33**4 * -8.482)**2/(1 - 0.33**2))
        sigma_E4He = np.sqrt(0.005 + (1 * 0.33**4 * -28.296)**2/(1 - 0.33**2))
        sigma_Rp4He = np.sqrt(0.0062 + (1 * 0.33**4 * 1.4552)**2/(1 - 0.33**2))
        sigma_fT3H = np.sqrt(3. + (1 * 0.33**4 * 1129.6)**2/(1 - 0.33**2))
        ax.text(-2.2, 0.7, 'With EFT Error \n $\\bar{c}=1$, $Q=0.33$', color='black', 
        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5'), fontsize=font_size)
    else:
        ax.text(-2.2, 0.78, 'No EFT Error', color='black', 
        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5'), fontsize=font_size)

    cs= ax.contourf(cD_eval, cE_eval, fT3H_eval, levels=[mean_fT3H-x*sigma_fT3H, mean_fT3H+x*sigma_fT3H], colors='gray', alpha=0.2)
    contour_lines = ax.contour(cs, levels=cs.levels[:3], colors='black')
    cs = ax.contourf(cD_eval, cE_eval, Rp4He_eval, levels=[mean_Rp4He-x*sigma_Rp4He, mean_Rp4He+x*sigma_Rp4He], colors='#ffb16d', alpha=0.85)
    contour_lines = ax.contour(cs, levels=cs.levels[:3], colors='#e17701')
    cs = ax.contourf(cD_eval, cE_eval, E3H_eval, levels=[mean_E3H-x*sigma_E3H, mean_E3H+x*sigma_E3H], colors='#7af9ab', alpha=0.8)
    contour_lines = ax.contour(cs, levels=cs.levels[:3], colors='#2baf6a', linestyles='-')
    cs = ax.contourf(cD_eval, cE_eval, E4He_eval, levels=[mean_E4He-x*sigma_E4He, mean_E4He+x*sigma_E4He], colors='dodgerblue', alpha=0.5)
    contour_lines = ax.contour(cs, levels=cs.levels[:3], colors='dodgerblue', linestyles='-')

    return ax

def customizer(ax):

    ax.set_xlabel('$c_D$', fontsize=font_size+4)
    ax.set_ylabel('$c_E$', fontsize=font_size+4)
    ax.text(0., -0.85, '$f$ $T_{1/2}$', color='black', 
        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5'), fontsize=font_size)
    ax.text(-2.2, -0.75, '$r$ ($^4$He)', color='#e17701', 
        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5'), fontsize=font_size)
    ax.text(1.7, 0.05, '$E$ ($^3$H)', color='#2baf6a', 
        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5'), fontsize=font_size)
    ax.text(-2.2, 0.12, '$E$ ($^4$He)', color='dodgerblue', 
        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5'), fontsize=font_size)
    ax.tick_params(axis='x', labelsize=font_size)
    ax.tick_params(axis='y', labelsize=font_size)

save = False
efterror = False

### Comment after one use for customization of text boxes by changing
### the customizer function
#ax = constraint_plotter(cD_eval, cE_eval, efterror=efterror)

for txt in ax.texts[1:]:
    txt.set_visible(False)
    ax.texts[1:].remove(txt)

customizer(ax)

fig = ax.get_figure()
if efterror and save:
    fig.savefig('figs/constraints_EFT.pdf', bbox_inches='tight')
elif save:
    fig.savefig('figs/constraints_noEFT.pdf', bbox_inches='tight')


In [118]:
import emcee
import scipy.stats as stats
import corner

sigma_E3H = np.sqrt(0.015 + (1 * 0.33**4 * -8.482)**2/(1 - 0.33**2))
sigma_E4He = np.sqrt(0.005 + (1 * 0.33**4 * -28.296)**2/(1 - 0.33**2))
sigma_Rp4He = np.sqrt(0.0062 + (1 * 0.33**4 * 1.4552)**2/(1 - 0.33**2))
sigma_fT3H = np.sqrt(3. + (1 * 0.33**4 * 1129.6)**2/(1 - 0.33**2))


def log_prior(params):
    
    cD, cE = params
    cD_mu = 0.
    cD_sigma = 5.
    cE_mu = 0.
    cE_sigma = 5.

    return stats.norm.logpdf(cD, loc=cD_mu, scale=cD_sigma) + stats.norm.logpdf(cE, loc=cE_mu, scale=cE_sigma)

def log_likelihood(cD, cE, data_set):
    E4He_eval, Rp4He_eval, _, _, E3H_eval, _, fT3H_eval = vfewnucleonEmulator(cD, cE)
    
    E4He_likelihood = np.random.normal(E4He_eval, sigma_E4He)
    Rp4He_likelihood = np.random.normal(Rp4He_eval, sigma_Rp4He)
    E3H_likelihood = np.random.normal(E3H_eval, sigma_E3H)
    fT3H_likelihood = np.random.normal(fT3H_eval, sigma_fT3H)

    if data_set == 1:
        return stats.norm.logpdf(loc=E4He_eval, scale=sigma_E4He) + stats.norm.logpdf(loc=Rp4He_eval, scale=sigma_Rp4He)
    elif data_set == 2:
        pass

def log_posterior(cD, cE, data_set):
    return log_prior() + log_likelihood(cD, cE, data_set)

cD = np.linspace(-2.5, 2.5, 100)
cE = np.linspace(-1, 1, 100)
params = [cD, cE]
x = log_prior(params)
y = log_likelihood(cD, cE, 1)

[-24.04569368 -24.80742989 -23.92841391 -24.52224779 -24.58970258
 -24.32933734 -24.8323118  -25.16991002 -25.39868746 -25.33433638
 -25.56961061 -25.7399182  -24.87351745 -25.85628966 -25.96532291
 -26.43490802 -26.91530967 -26.09273636 -25.99512723 -26.70330327
 -26.38806748 -26.22445792 -26.9694292  -27.71696123 -27.04198336
 -27.42330294 -27.3912653  -27.63052002 -27.46982826 -27.93185007
 -27.53248458 -28.16840921 -27.96243659 -28.59992336 -28.18377878
 -28.90353717 -28.17153088 -29.50942925 -28.90796291 -28.86352169
 -29.29206655 -29.85220514 -28.97247462 -29.58040791 -30.01964707
 -29.60057743 -30.07332709 -29.89204504 -30.62758206 -30.74010867
 -30.56901167 -30.58226097 -31.32063083 -31.67069955 -31.43598478
 -32.25870616 -31.70298463 -32.12931938 -32.55203483 -32.06591312
 -32.23457817 -32.36136707 -32.24556201 -33.43896302 -33.33355887
 -32.68118942 -32.79115412 -33.9001839  -33.51131652 -34.05708009
 -34.05265719 -34.08159078 -34.37404851 -34.17255434 -34.8168527
 -34.687484

/var/folders/3z/njryns916658jt56y6zm3xfw0000gn/T/ipykernel_35665/3912172925.py:32: RuntimeWarning: invalid value encountered in log
  return np.log(E4He_likelihood) + np.log(Rp4He_likelihood)
